In [1]:
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

Requirement already up-to-date: pyswarm in /usr/local/lib/python3.6/dist-packages (0.6)
Requirement already up-to-date: pactools in /usr/local/lib/python3.6/dist-packages (0.3)


In [2]:
from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical

import skimage.transform

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Running on PyMC3 v3.7


In [3]:
train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)

In [4]:
train4=np.empty([20260, 72,72,3], dtype=int)

In [5]:
for i in range(len(train1)):
  train4[i] = skimage.transform.resize(train1[i], (72, 72))

In [6]:
train2,test1, train_labels2,test_labels1 = train_test_split(train4, train_labels1, test_size=0.2, random_state=42)

In [7]:
train=train2/225.0
train_labels = pd.get_dummies(train_labels2)

test=test1/225.0
test_labels = pd.get_dummies(test_labels1)

In [8]:
IMG_SHAPE2=(72,72,3) 

xception=keras.applications.xception.Xception(input_shape=IMG_SHAPE2,
                                               include_top=False,
                                               weights='imagenet')

83689472/83683744 [==============================] - 1s 0us/step


In [9]:
#fine_tuning,flatvsglobalpool,dropout,dense_layers
lb=[0,0,0,0]
ub=[19,1,0.5,4]

In [10]:
def create_model_dense_xception(x): 
  tempex=xception
  for layer in tempex.layers[:(-1)*int(round(x[0]))]:
    layer.trainable = False
  
  #vgg19.trainable=False
  model = tf.keras.Sequential()
  model.add(tempex)
  if (int(round(x[1]))):
    model.add(keras.layers.Flatten())
  else:
    model.add(keras.layers.GlobalAveragePooling2D())
  if (int(round(x[3]))==4):
    layers=[256,128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==3):
    layers=[128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==2):
    layers=[64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==1):
    layers=[32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  else:
    layers=[]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))

  for layer in model.layers:
    print(layer, layer.trainable)
  model.add(keras.layers.Dense(3,activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
  return model

In [11]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')

In [12]:
def apple(x):
  model = create_model_dense_xcpetion(x)
  model.fit(train, train_labels, epochs=20, batch_size=1000, verbose=1,validation_data=(test, test_labels),callbacks=[EarlyStopper])
  loss, _ = model.evaluate(test, test_labels, verbose=1)
  return loss 

In [13]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))

<tensorflow.python.keras.engine.functional.Functional object at 0x7f1131d82780> True
<tensorflow.python.keras.layers.core.Flatten object at 0x7f1131d5fbe0> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f1131d792b0> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f11317a1d30> True
Epoch 1/20
17/17 [==============================] - 8s 479ms/step - loss: 0.8029 - accuracy: 0.6471 - val_loss: 1.5608 - val_accuracy: 0.7683
Epoch 2/20
17/17 [==============================] - 6s 364ms/step - loss: 0.3524 - accuracy: 0.8726 - val_loss: 0.5677 - val_accuracy: 0.8492
Epoch 3/20
17/17 [==============================] - 6s 364ms/step - loss: 0.2620 - accuracy: 0.9036 - val_loss: 0.6186 - val_accuracy: 0.8285
Epoch 4/20
17/17 [==============================] - 6s 364ms/step - loss: 0.2124 - accuracy: 0.9216 - val_loss: 0.4401 - val_accuracy: 0.8564
Epoch 5/20
17/17 [==============================] - 6s 365ms/step - loss: 0.1663 - accuracy: 0.9373 - val_loss: 0.3716 - 